# PyEDA

Наработки в использовании библиотеки [`pyeda`][1]. Решение содоку.

[1]: https://pyeda.readthedocs.io/en/latest/sudoku.html

In [1]:
import os
from pyeda.inter import *

In [2]:
DIGITS = "123456789"

In [3]:
a, b, c = map(exprvar, 'abc')

In [4]:
f = OneHot(a, b, c)
f

And(Or(~a, ~b), Or(~a, ~c), Or(~b, ~c), Or(a, b, c))

In [5]:
f.to_dnf()

Or(And(a, ~b, ~c), And(~a, b, ~c), And(~a, ~b, c))

In [6]:
expr2truthtable(f)

c b a
0 0 0 : 0
0 0 1 : 1
0 1 0 : 1
0 1 1 : 0
1 0 0 : 1
1 0 1 : 0
1 1 0 : 0
1 1 1 : 0

In [7]:
fbdd = expr2bdd(f)

In [8]:
imgaesDir = 'img'
imageName = 'graphviz_f'
imageFormat = 'svg'
fileName = '{0}.{1}'.format(imageName, imageFormat)
filePath = os.path.join(imgaesDir, fileName)
# Удалите файл если необходимо его пересоздать
if not os.path.isfile(filePath):
    from graphviz import Source
    gv = Source(fbdd.to_dot())
    gv.render(imageName, directory=imgaesDir, cleanup=True, format=imageFormat)

![BDE](img/graphviz_f.svg "Диаграмма решений")

In [9]:
X = exprvars('y', (1, 10))

In [10]:
[ X[v] for v in range(1, 10) ]

[y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9]]

In [11]:
V = And(*[ X[v] for v in range(1, 10) ])
V

And(y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9])

In [12]:
X = exprvars('x', (1, 10), (1, 10), (1, 10))

> every square on the board can assume only one value

In [13]:
V = And(*[
        And(*[
            OneHot(*[ X[r, c, v]
                for v in range(1, 10) ])
            for c in range(1, 10) ])
        for r in range(1, 10) ])

> every square in each row is unique

In [14]:
R = And(*[
        And(*[
            OneHot(*[ X[r, c, v]
                for c in range(1, 10) ])
            for v in range(1, 10) ])
        for r in range(1, 10) ])

> every square in each column is unique

In [15]:
C = And(*[
        And(*[
            OneHot(*[ X[r, c, v]
                for r in range(1, 10) ])
            for v in range(1, 10) ])
        for c in range(1, 10) ])

> every square in a box is unique

In [16]:
B = And(*[
        And(*[
            OneHot(*[ X[3*br+r, 3*bc+c, v]
                for r in range(1, 4) for c in range(1, 4) ])
            for v in range(1, 10) ])
        for br in range(3) for bc in range(3) ])

In [17]:
S = And(V, R, C, B)
len(S.xs)

11988

In [18]:
def parse_grid(grid):
    chars = [c for c in grid if c in DIGITS or c in "0."]
    assert len(chars) == 9 ** 2
    return And(*[
        X[i // 9 + 1, i % 9 + 1, int(c)]
        for i, c in enumerate(chars) if c in DIGITS
    ])

In [23]:
grid = ( ".73|...|8.."
         "..4|13.|.5."
         ".85|..6|31."
         "---+---+---"
         "5..|.9.|.3."
         "..8|.1.|5.."
         ".1.|.6.|..7"
         "---+---+---"
         ".51|6..|28."
         ".4.|.52|9.."
         "..2|...|64." )

In [24]:
def get_val(point, r, c):
    for v in range(1, 10):
        if point[X[r, c, v]]:
            return DIGITS[v-1]
    return "X"

def display(point):
    chars = list()
    for r in range(1, 10):
        for c in range(1, 10):
            if c in (4, 7):
                chars.append("|")
            chars.append(get_val(point, r, c))
        if r != 9:
            chars.append("\n")
            if r in (3, 6):
                chars.append("---+---+---\n")
    print("".join(chars))

In [25]:
def solve(grid):
     with parse_grid(grid):
         return S.satisfy_one()

In [26]:
display(solve(grid))

173|529|864
694|138|752
285|476|319
---+---+---
567|294|138
428|713|596
319|865|427
---+---+---
951|647|283
846|352|971
732|981|645
